In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

## Load and Pre-process Data

In [2]:
# Load data
file_path = './filtered_columns_data/'

raw_data2016 = pd.read_csv(file_path + 'LCA_FY2016.csv')
raw_data2016 = raw_data2016.rename({'CASE_SUBMITTED': 'RECEIVED_DATE', 'SOC_NAME': 'SOC_TITLE'}, axis=1)

raw_data2017 = pd.read_csv(file_path + 'LCA_FY2017.csv')
raw_data2017 = raw_data2017.rename({'CASE_SUBMITTED': 'RECEIVED_DATE', 'SOC_NAME': 'SOC_TITLE'}, axis=1)

raw_data2018 = pd.read_csv(file_path + 'LCA_FY2018.csv')
raw_data2018 = raw_data2018.rename({'CASE_SUBMITTED': 'RECEIVED_DATE', 'SOC_NAME': 'SOC_TITLE'}, axis=1)

raw_data2020 = pd.concat(
    map(pd.read_csv, [(file_path+'LCA_FY2020_Q1.csv'), (file_path+'LCA_FY2020_Q2.csv'), 
                      (file_path+'LCA_FY2020_Q3.csv'), (file_path+'LCA_FY2020_Q4.csv')]), ignore_index=True)

raw_data2021 = pd.concat(
    map(pd.read_csv, [(file_path+'LCA_FY2021_Q1.csv'), (file_path+'LCA_FY2021_Q2.csv'), 
                      (file_path+'LCA_FY2021_Q3.csv'), (file_path+'LCA_FY2021_Q4.csv')]), ignore_index=True)

/Users/soomi/anaconda3/lib/python3.7/site-packages/pandas/core/reshape/concat.py:304: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  sort=sort,


In [3]:
# Job occupation codes and corresponding names
big_group_dict = {
    11: 'Management Occupations',
    13: 'Business and Financial Operations Occupations',
    15: 'Computer and Mathematical Occupations',
    17: 'Architecture and Engineering Occupations',
    19: 'Life, Physical, and Social Science Occupations',
    21: 'Community and Social Service Occupations',
    23: 'Legal Occupations',
    25: 'Educational Instruction and Library Occupations',
    27: 'Arts, Design, Entertainment, Sports, and Media Occupations',
    29: 'Healthcare Practitioners and Technical Occupations',
    31: 'Healthcare Support Occupations',
    33: 'Protective Service Occupations',
    35: 'Food Preparation and Serving Related Occupations',
    37: 'Building and Grounds Cleaning and Maintenance Occupations',
    39: 'Personal Care and Service Occupations',
    41: 'Sales and Related Occupations',
    43: 'Office and Administrative Support Occupations',
    45: 'Farming, Fishing, and Forestry Occupations',
    47: 'Construction and Extraction Occupations',
    49: 'Installation, Maintenance, and Repair Occupations',
    51: 'Production Occupations',
    53: 'Transportation and Material Moving Occupations',
    55: 'Military Specific Occupations'
}

In [4]:
# Pre-process loaded data
def get_data(data):
    
    data = data.rename({'CASE_SUBMITTED': 'RECEIVED_DATE', 'SOC_NAME': 'SOC_TITLE'}, axis=1)
    
    # Deal with erroneous data in SOC_CODE
    data = data[data['SOC_CODE'].notna()]
    data = data.replace({np.nan: None})
    data = data[~data['SOC_CODE'].str.contains("[a-zA-Z]").fillna(False)]
    data = data[data['SOC_CODE'].apply(lambda x: len(x.split('-')[0])>1)]
    
    # Big group define
    data['BIG_GROUP_CODE'] = [int(str(i)[0:2]) for i in data.SOC_CODE]
    data = data[data['BIG_GROUP_CODE'].isin(big_group_dict.keys())] # drop rows with weird SOC codes
    data['BIG_GROUP_NAME'] = [big_group_dict[i] for i in data.BIG_GROUP_CODE]
    
    data['WAGE_RATE_OF_PAY_FROM'] = [convert_wage_str(i) for i in data.WAGE_RATE_OF_PAY_FROM]
    data['WAGE_RATE_OF_PAY_TO'] = [convert_wage_str(i) for i in data.WAGE_RATE_OF_PAY_TO]
    
    data = data.replace({np.nan: None})
    
    # Calculate average wage
    tmp = []
    for idx, row in data.iterrows():
        if (row.WAGE_RATE_OF_PAY_TO) is None:
            tmp.append(row.WAGE_RATE_OF_PAY_FROM)
        else:
            tmp.append((row.WAGE_RATE_OF_PAY_FROM + row.WAGE_RATE_OF_PAY_TO) / 2)
            
    data['AVERAGE_WAGE'] = tmp
    
    # Uppercase CASE_STATUS & EMPLOYER_NAME
    data['CASE_STATUS'] = data['CASE_STATUS'].str.upper()
    data['EMPLOYER_NAME'] = data['EMPLOYER_NAME'].str.upper()
    
    # Convert datetime
    data.RECEIVED_DATE = pd.to_datetime(data.RECEIVED_DATE)
    data.DECISION_DATE = pd.to_datetime(data.DECISION_DATE)
    data.ORIGINAL_CERT_DATE = pd.to_datetime(data.ORIGINAL_CERT_DATE)
    
    return data

def convert_wage_str(wage_str):
    if wage_str is None:
        return None
    
    # Delete the dollar sign
    wage_str = wage_str[1:]
    
    wage_str = wage_str.replace(',', '')
    return float(wage_str)

In [5]:
# Pre-process data for each year
data2016 = get_data(raw_data2016)
data2017 = get_data(raw_data2017)
data2018 = get_data(raw_data2018)
data2020 = get_data(raw_data2020)
data2021 = get_data(raw_data2021)

## Filter data by company and visualize relevant results

In [6]:
# Filter data of the particular company only
def filter_data_by_company_name(data, company_name):
    return data[(data.EMPLOYER_NAME.str.contains(company_name)) & (data.VISA_CLASS == 'H-1B')]

def get_company_data(company_name):

    company_data_2016 = filter_data_by_company_name(data2016, company_name)
    company_data_2017 = filter_data_by_company_name(data2017, company_name)
    company_data_2018 = filter_data_by_company_name(data2018, company_name)
    company_data_2020 = filter_data_by_company_name(data2020, company_name)
    company_data_2021 = filter_data_by_company_name(data2021, company_name)
    
    # Compile the data over the years
    data = {
        2016: company_data_2016,
        2017: company_data_2017,
        2018: company_data_2018,
        2020: company_data_2020,
        2021: company_data_2021
    }
    
    return data

### Visualize the approval rate and waiting time

In [7]:
def approval_number(data):
    return len(data[data.CASE_STATUS == 'CERTIFIED']) + len(data[data.CASE_STATUS == 'CERTIFIED-WITHDRAWN'])

def approval_rate(data):
    return approval_number(data) / len(data)

def denial_number(data):
    return len(data[data.CASE_STATUS == 'DENIED'])

def calculate_approval_rate_over_years(data):
    approval_rate_over_years = []
    years = []
    for year, data_year in data.items():
        approval_rate_res = approval_rate(data_year)
        years.append(year)
        approval_rate_over_years.append(approval_rate_res)
    return approval_rate_over_years, years

def plot_approval_rate_over_years(data):
    approval_rate_over_years, years = calculate_approval_rate_over_years(data)
    plt.plot(years, approval_rate_over_years)
    plt.title('Approval rate over years')
    plt.show()

def calculate_waiting_time(data):
    result = []
    for index, row in data.iterrows():
        if row.CASE_STATUS == 'CERTIFIED-WITHDRAWN':
            result.append((row.ORIGINAL_CERT_DATE - row.RECEIVED_DATE).days)
        elif row.CASE_STATUS == 'CERTIFIED':
            result.append((row.DECISION_DATE - row.RECEIVED_DATE).days)
    return np.mean(result)

def plot_waiting_time(data):
    result, years = [], []
    for year, data_year in data.items():
        years.append(year)
        result.append(calculate_waiting_time(data_year))
    plt.plot(years, result)
    plt.title('Waiting time over years')
    plt.show()

### Visualize the number of applications per job industry

In [8]:
def plot_different_groups_applications_number(data, big_group_dict):
    # Find all uniques groups inside the data
    unique_groups = set(company_data_2021.BIG_GROUP_CODE)
    # Results
    applications_count = []
    
    # For each group, count how many applications
    for group_code in unique_groups:
        count = count_applications_on_CODE(group_code, data)
        applications_count.append(count)
        
    # Present the results
    fig = plt.figure()
    ax = fig.add_axes([0,0,1,1])
    groups = [big_group_dict[i] for i in unique_groups] 
    ax.barh(groups, applications_count)
    for i, v in enumerate(applications_count):
        ax.text(v + 3, i + .25, str(v), color='blue', fontweight='bold')

    plt.title("Number of applications per job industry")
    plt.show()
    
def count_applications_on_CODE(group_code, data):
    return sum(data.BIG_GROUP_CODE == group_code)

### Visualize the wage data per job

In [9]:
def plot_jobs_on_group(group_code, data):
    # Filter all applications with the same group code
    data = data.loc[data.BIG_GROUP_CODE == group_code]
    
    # Find all uniques job_code
    unique_job_titles = set(data.SOC_TITLE)
    
    # Get the job title based on job_code - later, if necessary
   
    # Calculate salary stats for each job code
    average_on_job = []
    jobs = []
    for job_title in unique_job_titles:
        jobs.append(job_title)
        average_on_job.append(calculate_salary_stats(job_title, data))
    # Plot
    fig = plt.figure()
    ax = fig.add_axes([0,0,1,1])
    ax.barh(jobs, average_on_job)
    for i, v in enumerate(average_on_job):
        ax.text(v + 3, i - .1 , str(v), color='blue', fontweight='bold')

    plt.title("Average wage based on job")
    plt.show()
    

def calculate_salary_stats(job_title, data):
    # Find all applications of specific job code
    new_data = data.loc[data.SOC_TITLE == job_title]
    # Collect all salaries
    salaries = new_data.AVERAGE_WAGE
    
    # Mean
    mean_salaries = np.round(np.mean(salaries))
    
    # Median
    median_salaries = np.round(np.median(salaries))
    
    # Min, Max
    min_salaries = min(salaries)
    max_salaries = max(salaries)
    
    # 95 Confidence Interval
    
    return mean_salaries

## Find companies that have consistent names throughout the years

In [10]:
# find the unique company names per data
def unique_companies(data):
    data['EMPLOYER_NAME'] = data['EMPLOYER_NAME'].str.upper()
    return data['EMPLOYER_NAME'].unique()

companies_2016 = unique_companies(raw_data2016)
companies_2017 = unique_companies(raw_data2017)
companies_2018 = unique_companies(raw_data2018)
companies_2020 = unique_companies(raw_data2020)
companies_2021 = unique_companies(raw_data2021)

In [11]:
# find companies with overlapping names over the years
overlap_2020_2021 = []
for company in companies_2021:
    if company in companies_2020:
        overlap_2020_2021.append(company)
print(f"{len(overlap_2020_2021)} companies have the same name in 2020 and 2021")

overlap_2018_2020 = []
for company in overlap_2020_2021:
    if company in companies_2018:
        overlap_2018_2020.append(company)
print(f"{len(overlap_2018_2020)} companies have the same name in 2018 and 2020")

overlap_2017_2018 = []
for company in overlap_2018_2020:
    if company in companies_2017:
        overlap_2017_2018.append(company)
print(f"{len(overlap_2017_2018)} companies have the same name in 2017 and 2018")

overlap_2016_2017 = []
for company in overlap_2017_2018:
    if company in companies_2016:
        overlap_2016_2017.append(company)
print(f"{len(overlap_2016_2017)} companies have the same name in 2016 and 2017")

27160 companies have the same name in 2020 and 2021
14712 companies have the same name in 2018 and 2020
11441 companies have the same name in 2017 and 2018
9183 companies have the same name in 2016 and 2017


In [12]:
overlapping_companies = overlap_2016_2017
overlapping_companies.pop(3647) # remove erroneous entry

'RETAIL SERVICES AND SYSTEMS, INC. (DBA TOTAL WINE'

## Retrieve search filter data for each company (only using 2021 data)

In [13]:
def get_stats_per_company(company_name):
    data = filter_data_by_company_name(data2021, company_name)
    columns = ['EMPLOYER_CITY', 'EMPLOYER_STATE', 'BIG_GROUP_NAME', 'FULL_TIME_POSITION']
    result = {}
    for col in columns:
        result[col] = get_unique_values(data, col)
    job_industry_list = result['BIG_GROUP_NAME']
    result['JOB_INDUSTRY_WAGE'] = create_job_industry_and_wage(data, job_industry_list)
    result['EMPLOYER_CITY'] = validate_city(result['EMPLOYER_CITY'], result['EMPLOYER_STATE'])
    result['FULL_TIME_POSITION'] = convert_FT_PT(result['FULL_TIME_POSITION'])
    result['COMPANY_NAME'] = company_name
    result['APPROVAL_RATE'] = create_job_industry_and_approval_rate(data, job_industry_list)
    
    return result

def get_unique_values(data, column):
    return list(np.unique(data[column].values))

### Validate city names

In [14]:
# Evaluate the validity of the city names in the dataset
# Ignore data entries with erroneous city names (ex. typos)
def create_cities_on_states():
    states_cities_df = pd.read_csv('./us_cities_states_counties.csv', delimiter='|', on_bad_lines='skip')
    cities_on_states = {}
    for index, row in states_cities_df.iterrows():
        city = row[0]
        state = row[1]
        if state not in cities_on_states:
            cities_on_states[state] = set()
        cities_on_states[state].add(city)
    return cities_on_states

cities_on_states = create_cities_on_states()

def get_legit_cities(cities_on_states, state_list):
    res = []
    for state in state_list:
        res += (cities_on_states[state])
    return res

def validate_city(city_list, state_list):
    # get a list of the legit cities
    legit_cities = set(get_legit_cities(cities_on_states, state_list))
    
    # make every city name in the same format (first letters are capitalized)
    # and get unique city values
    check_cities = set([city.title() for city in city_list])
    
    # check whether the city in dataset is valid by finding matches
    valid_cities = check_cities.intersection(legit_cities)
    
    # return a list of cities that are in valid form
    return list(valid_cities)

### Calculate the average and range of salary per job industry

In [15]:
def calculate_wage_per_job_industry(data, job_industry_name):
    
    # ignore entries with hourly wages
    data = data[(data['WAGE_UNIT_OF_PAY']=='Year') & (data['BIG_GROUP_NAME']==job_industry_name)]
    
    if len(data)==0:
        return 0
    
    # get the minimum & mean & maximum of AVERAGE_WAGE
    salaries = data.AVERAGE_WAGE
    
    # Mean
    mean_salaries = np.round(np.mean(salaries))
    
    # Min, Max
    min_salaries = min(salaries)
    max_salaries = max(salaries)
    
    return [mean_salaries, min_salaries, max_salaries]

def create_job_industry_and_wage(data, job_industry_list):
    result = {}
    for job_industry in job_industry_list:
        result[job_industry] = calculate_wage_per_job_industry(data, job_industry)
    return result

### Calculate the 2021 application approval rate per job industry

In [16]:
def calculate_approval_rate_per_job_industry(data, job_industry_name):
    
    data = data[(data['BIG_GROUP_NAME']==job_industry_name)]
    
    return approval_rate(data)

def create_job_industry_and_approval_rate(data, job_industry_list):
    result = {}
    for job_industry in job_industry_list:
        result[job_industry] = calculate_approval_rate_per_job_industry(data, job_industry)
    return result

### Convert Full-Time vs Part-Time entries into relevant formats

In [17]:
# [Y, N] => ["Full-time, Part-time]
def convert_FT_PT(FT_PT_data):
    for i in range(len(FT_PT_data)):
        if FT_PT_data[i] == "Y":
            FT_PT_data[i] = "Full-time"
        else:
            FT_PT_data[i] = "Part-time"
            
    return FT_PT_data

In [18]:
company_name_list = []
city_list = []
state_list = []
job_industry_list = []
full_time_list = []
approval_rate_list = []

for company in overlapping_companies:
    result = get_stats_per_company(company)
    
    company_name_list.append(result['COMPANY_NAME'])
    city_list.append(result['EMPLOYER_CITY'])
    state_list.append(result['EMPLOYER_STATE'])
    job_industry_list.append(result['JOB_INDUSTRY_WAGE'])
    full_time_list.append(result['FULL_TIME_POSITION'])
    approval_rate_list.append(result['APPROVAL_RATE'])

/Users/soomi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  This is separate from the ipykernel package so we can avoid doing imports until


TypeError: '<' not supported between instances of 'str' and 'NoneType'

In [ ]:
result_df = pd.DataFrame({'Company_Name': company_name_list, 'City': city_list, 'State': state_list, 
                   'Job_Industry': job_industry_list, 'Full_Time': full_time_list})

In [ ]:
result_df.to_csv('company_search_filters.csv', index=False)

In [ ]:
test_df = pd.read_csv('company_search_filters.csv')
test_df